## Homework 05: First Order ODEs
### PHYS420 — Intro to Computational Physics — Fall 2025  
### Hayden Dauphin

<img src="./Images/hw5q1.png" width="700"/>

In [32]:
# --- Dependencies --- 
import matplotlib.pyplot as plt
import numpy as np
import scipy as sci 
from scipy.integrate import solve_ivp

In [33]:
# ---- Problem 1: Coding and testing ----

# --- Code 1: Explicit Euler --- 

def explicit_euler(f, timeInterval, initCond, stepSize):
    ''' 
    Function to compute first order ODEs with the explicit Euler method
    Arguments: 
        f: function 
        timeInterval: interval of time to be evaluated over
        initCond: initial condition at start of timeInterval 
        stepSize: size of each step t_{i+1}
    Returns: 
        t_vals: array of time values 
        y_vals: array of solution values at each time value
    '''

    t_start, t_stop = timeInterval  
    num_steps = int((t_stop - t_start) / stepSize)  # calculate number of steps by dividing length of interval by size of steps 
    t_vals = np.linspace(t_start, t_stop, num_steps + 1)    # create array of time values 
    y_vals = np.zeros_like(t_vals, dtype=float)  # create array of zeros with length = len(t_vals) and data type = type(t_vals)
    y_vals[0] = initCond    # set initial condition as first y value 

    # -- Loop to calculate solutions at each time point with equation for explicit Euler: y_(i+1) = y_i + h * f(t_i, y_i)
    for i in range(num_steps):
        y_vals[i+1] = y_vals[i] + stepSize * f(t_vals[i], y_vals[i])
    
    return t_vals, y_vals

# -- Test function for explicit Euler --
print("Explicit Euler test: ")
f1 = lambda t, y: -y 
times1, sols1 = explicit_euler(f1, (0,5), 1.0, 0.001)
print("Final y = ", sols1[-1])
print("Exact = ", np.exp(-5))
percenterr = (np.abs((np.exp(-5) - sols1[-1])/np.exp(-5))*100)
print("Percent error: ", percenterr, "%")


Explicit Euler test: 
Final y =  0.006721111959865623
Exact =  0.006737946999085467
Percent error:  0.24985413542328475 %


In [34]:
# --- Code 2: RKF45 --- 

# - Fehlberg 4(5) coefficients - 
c = [0, 1/4, 3/8, 12/13, 1, 1/2]
a = [
    [0,0,0,0,0],
    [1/4,0,0,0,0],
    [3/32,9/32,0,0,0],
    [1932/2197,-7200/2197,7296/2197,0,0],
    [439/216,-8,3680/513,-845/4104,0],
    [-8/27,2,-3544/2565,1859/4104,-11/40]
]
# - 4th and 5th order weights -
b5 = [16/135,0,6656/12825,28561/56430,-9/50,2/55]
b4 = [25/216,0,1408/2565,2197/4104,-1/5,0]

def rkf45(f, t0, tf, y0, relTol=1e-8, absTol=1e-10, h=0.01): 
    ''' 
    RKF45 method of solving first order ODEs
    Arguments: 
        f: function 
        t: time interval 
        y0: initial condition 
        relTol: relative tolerance 
        absTol: absolute tolerance 
        h: initial step size 
    Returns: 
        ts: array of the time values solution computed for 
        ys: array of solutions 
        errs: array of error estimates for each accepted step 
        n_accept: number of accepted 
        n_reject: number of rejected steps 
    '''
    t = t0 
    y = np.array(y0, dtype=float)
    ts = [t]
    ys = [y.copy()]
    errs = [0.0]
    n_accept = n_reject = 0 


    # failsafe counter
    max_steps = 100000
    count = 0

    while t < tf and count < max_steps: 
        if t + h > tf: 
            h = tf - t 
        
        #compute k 1-6 
        k1 = f(t, y)
        k2 = f(t + c[1]*h, y + h*a[1][0]*k1)
        k3 = f(t + c[2]*h, y + h*(a[2][0]*k1 + a[2][1]*k2))
        k4 = f(t + c[3]*h, y + h*(a[3][0]*k1 + a[3][1]*k2 + a[3][2]*k3))
        k5 = f(t + c[4]*h, y + h*(a[4][0]*k1 + a[4][1]*k2 + a[4][2]*k3 + a[4][3]*k4))
        k6 = f(t + c[5]*h, y + h*(a[5][0]*k1 + a[5][1]*k2 + a[5][2]*k3 + a[5][3]*k4 + a[5][4]*k5))

        Ks = np.array([k1, k2, k3, k4, k5, k6])
        y5 = y + h * np.dot(b5, Ks)
        y4 = y + h * np.dot(b4, Ks)
        err = np.linalg.norm(y5 - y4)

        # scaled error test
        scale = absTol + relTol * np.maximum(np.abs(y), np.abs(y5))
        err_norm = err/np.linalg.norm(scale)

        count += 1

        if err_norm <= 1.0: 
            t += h
            y = y5 
            ts.append(t)
            ys.append(y.copy())
            errs.append(err_norm)
            n_accept += 1
        else: 
            n_reject += 1 

        # update step size 
        if err_norm == 0: 
            scaling = 2.0 
        else: 
            scaling = 0.9 * err_norm**(-0.2) 
        scaling = np.clip(scaling, 0.2, 5.0)
        h *= scaling 
    if count >= max_steps:
        print("Warning: maximum number of steps reached. Integration might be unfinished.")
    return np.array(ts), np.array(ys), np.array(errs), n_accept, n_reject

print("\nRKF45 test:")

f2 = lambda t2, y2: y2 *(1 - y2)
times2, sols2, errEst, acc, rej = rkf45(f2, 0, 10, 0.1, h=0.001)
print(f"Final y = {sols2[-1]:.6f}, expected = ~1.0, accepted = {acc}, rejected = {rej}, max err = {np.max(errEst):.2e}")


RKF45 test:
Final y = 0.999592, expected = ~1.0, accepted = 58, rejected = 2, max err = 9.18e-01


In [66]:
# --- Code 3: Library Routine ---

f3 = lambda t, y: -y 
sols3 = solve_ivp(f3, t_span=(0,5), y0=[1.0], method="RK45", rtol=1e-8, atol=1e-10)

print("Final y = ", sols3.y[0,-1])
print("Expected: ", np.exp(-5))
print("Steps taken: ", len(sols3.t))

Final y =  0.006737947094324274
Expected:  0.006737946999085467
Steps taken:  54


<img src="./Images/hw5q2a.png" width="700px"/>
<img src="./Images/hw5q2b.png" width="700px">

In [70]:
''' 
Variable names: 
    T_: time array
    Y_: solution array
    E_: error estimates 
    A_/R_: accepted/rejected counts
    sol_scipy*: scipy object, can access .t and .y for times and solutions 
'''

# Problem 2.1
def f21(t, x):
    return x + np.exp(-t) 

t21 = 1
exact21 = np.sinh(t21) # analytical solution at t=1

# x(0) = 0, t[0, 1]
T_euler21, Y_euler21 = explicit_euler(f21, (0,1), 0, 0.001)
T_rkf21, Y_rkf21, E_rkf21, A_rkf21, R_rkf21 = rkf45(f21, 0, 1, 0)
sol_scipy21 = solve_ivp(f21, (0,1), [0], method="RK45", rtol=1e-8, atol=1e-10)

# Problem 2.2
def f22(t, x):
    return x+2*np.cos(t) 

t22 = 1
exact22 = -np.cos(t22) + np.sin(t22) + 2*np.exp(t22)

# x(0) = 1, t[0, 1]
T_euler22, Y_euler22 = explicit_euler(f22, (0,1), 1, 0.001)
T_rkf22, Y_rkf22, E_rkf22, A_rkf22, R_rkf22 = rkf45(f22, 0, 1, 1)
sol_scipy22 = solve_ivp(f22, (0,1), [1], method="RK45", rtol=1e-8, atol=1e-10)


# Problem 2.3 
def f23(t, x):
    return t*(x**2)

t23 = 1
exact23 = 1/(1-(t23**2/2))

# x(0) = 1, t[0, 1]
T_euler23, Y_euler23 = explicit_euler(f23, (0,1), 1, 0.001)
T_rkf23, Y_rkf23, E_rkf23, A_rkf23, R_rkf23 = rkf45(f23, 0, 1, 1)
sol_scipy23 = solve_ivp(f23, (0,1), [1], method="RK45", rtol=1e-8, atol=1e-10)

# Problem 2.4
def f24(t, x):
    return 1.5*np.sin(2*x) - x*np.cos(t)

# x(0) = 1, t[0, 10]
T_euler24, Y_euler24 = explicit_euler(f24, (0,10), 1, 0.001)
T_rkf24, Y_rkf24, E_rkf24, A_rkf24, R_rkf24 = rkf45(f24, 0, 10, 1)
sol_scipy24 = solve_ivp(f24, (0,10), [1], method="RK45", rtol=1e-8, atol=1e-10)

# Problem 2.5 
def f25(t, x):
    return -0.25*x - x**3 + 1.2*np.sin(t)

# x(0) = 0, t[0,40], default A = 1.2
T_euler25, Y_euler25 = explicit_euler(f25, (0,40), 0, 0.001)
T_rkf25, Y_rkf25, E_rkf25, A_rkf25, R_rkf25 = rkf45(f25, 0, 40, 0)
sol_scipy25 = solve_ivp(f25, (0,40), [0], method="RK45", rtol=1e-8, atol=1e-10)

In [81]:
def compare_solvers(name, y_exact, T_euler, Y_euler, T_rkf, Y_rkf, sol_scipy):
    print(f"\n--- {name} ---")
    print(f"Euler final: {Y_euler[-1]:.6f}")
    print(f"RKF45 final: {Y_rkf[-1]:.6f}")
    print(f"SciPy final: {sol_scipy.y[0, -1]:.6f}")
    print(f"Exact:       {y_exact:.6f}")
    print(f"Euler error: {abs(Y_euler[-1]-y_exact):.3e}")
    print(f"RKF45 error: {abs(Y_rkf[-1]-y_exact):.3e}")
    print(f"SciPy error: {abs(sol_scipy.y[0, -1]-y_exact):.3e}")


compare_solvers("2.1", exact21, T_euler21, Y_euler21, T_rkf21, Y_rkf21, sol_scipy21)
compare_solvers("2.2", exact22, T_euler22, Y_euler22, T_rkf22, Y_rkf22, sol_scipy22)
compare_solvers("2.3", exact23, T_euler23, Y_euler23, T_rkf23, Y_rkf23, sol_scipy23)


--- 2.1 ---
Euler final: 1.174816
RKF45 final: 1.175201
SciPy final: 1.175201
Exact:       1.175201
Euler error: 3.853e-04
RKF45 error: 2.874e-09
SciPy error: 1.786e-09

--- 2.2 ---
Euler final: 5.734596
RKF45 final: 5.737732
SciPy final: 5.737732
Exact:       5.737732
Euler error: 3.136e-03
RKF45 error: 1.605e-08
SciPy error: 8.805e-09

--- 2.3 ---
Euler final: 1.996045
RKF45 final: 2.000000
SciPy final: 2.000000
Exact:       2.000000
Euler error: 3.955e-03
RKF45 error: 1.988e-07
SciPy error: 3.461e-08


In [82]:
def compare_vs_scipy(name, T_e, Y_e, T_r, Y_r, sol):
    ref = sol.y[0,-1]
    print(f"\n--- {name} (vs SciPy) ---")
    print(f"Euler final: {Y_e[-1]:.6f}")
    print(f"RKF45 final: {Y_r[-1]:.6f}")
    print(f"SciPy final: {ref:.6f}")
    print(f"Euler Error: {abs(Y_e[-1]-ref):.3e}")
    print(f"RKF Error: {abs(Y_r[-1]-ref):.3e}")

compare_vs_scipy("2.4", T_euler24, Y_euler24, T_rkf24, Y_rkf24, sol_scipy24)
compare_vs_scipy("2.5", T_euler25, Y_euler25, T_rkf25, Y_rkf25, sol_scipy25)



--- 2.4 (vs SciPy) ---
Euler final: 3.185228
RKF45 final: 3.187899
SciPy final: 3.187898
Euler Error: 2.670e-03
RKF Error: 7.222e-07

--- 2.5 (vs SciPy) ---
Euler final: 0.883782
RKF45 final: 0.883623
SciPy final: 0.883623
Euler Error: 1.585e-04
RKF Error: 4.099e-09
